<!-- Assignment 7 - SS 2023 -->

# Normalising Flows (15 points)

This notebook contains one of the assignments for the exercises in Deep Learning and Neural Nets 2.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility functions that should work without (too much) problems.
Please, do not alter this code or add extra import statements in your submission, unless explicitly allowed!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, we take a look at the change of variables formula
for distributions and how it can be used to train a generative model.
These generative models are known as normalising flows,
and make it possible to do density estimation with neural networks.

In [44]:
import torch
from torch import nn
from torch import optim
from torch import distributions
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from PIL import Image

torch.manual_seed(1806)
torch.cuda.manual_seed(1806)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [45]:
# google colab data management
import os.path

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    _home = 'gdrive/MyDrive/'
except ImportError:
    _home = '~'
finally:
    data_root = os.path.join(_home, '.pytorch')

print(data_root)

~\.pytorch


In [46]:
def data_to_image(*data: torch.Tensor,
                  means: tuple = (0, ), stds: tuple = (1., )) -> Image:
    """
    Convert multiple tensors to one big image.

    Parameters
    ----------
    data0, data1, ... dataN : torch.Tensor
        One or more tensors to be merged into a single image.
    means : tuple or torch.Tensor, optional
        Original mean of the image before normalisation.
    stds : tuple or torch.Tensor, optional
        Original standard deviation of the image before normalisation.

    Returns
    -------
    image : Image
        PIL image with all of the tensors next to each other.
    """
    # concatenate all data
    big_pic = torch.cat([x for x in data], dim=-1)

    means = torch.tensor(means)
    stds = torch.tensor(stds)
    to_image = transforms.Compose([
        # inverts normalisation of image
        transforms.Normalize(-means / stds, 1. / stds),
        transforms.Lambda(lambda x: torch.clamp(x, 0, 1)),
        transforms.ToPILImage()
    ])

    return to_image(big_pic)

In [47]:
class NormalisingFlowTrainer:
    
    def __init__(self,
        model: nn.Module,
        criterion: nn.Module,
        optimiser: optim.Optimizer,
    ):
        """
        Parameters
        ----------
        model : torch.nn.Module
            Neural Network that will be trained.
        criterion : torch.nn.Module
            Loss function to use for training.
        optimiser : torch.optim.Optimizer
            Optimisation strategy for training.
        tracker : Tracker, optional
            Tracker to keep track of training progress.
        """
        self.model = model
        self.criterion = criterion
        self.optimiser = optimiser

    def state_dict(self):
        """ Current state of learning. """
        return {
            "model": self.model.state_dict(),
            "objective": self.criterion.state_dict(),
            "optimiser": self.optimiser.state_dict(),
        }
    
    def load_state_dict(self, state_dict: dict):
        """ Restore learning state. """
        self.model.load_state_dict(state_dict["model"])
        self.criterion.load_state_dict(state_dict["objective"])
        self.optimiser.load_state_dict(state_dict["optimiser"])
        
    @property
    def device(self):
        """ Device of the (first) model parameters. """
        return next(self.model.parameters()).device

    def _forward(self, data: DataLoader, metric: callable):
        """ Unsupervised variation on forward propagation. """
        device = self.device

        for x, _ in data:
            x = x.to(device)
            outputs = self.model(x)
            res = metric(outputs)
            yield res

    @torch.no_grad()
    def evaluate(self, data: DataLoader, metric: callable) -> list:
        self.model.eval()

        results = self._forward(data, metric)
        return [res.item() for res in results]


    @torch.enable_grad()
    def update(self, data: DataLoader) -> list:
        self.model.train()

        errs = []
        for err in self._forward(data, self.criterion):
            errs.append(err.item())

            self.optimiser.zero_grad()
            err.backward()
            self.optimiser.step()

        return errs

    @torch.no_grad()
    def display_inverted(self, y: torch.Tensor, 
                         prior : distributions.Distribution = None):
        """
        Display data using the inverse function of a normalising flow.

        Parameters
        ----------
        flow : nn.Module
            The normalising flow.
        y : torch.Tensor
            A sample batch from the easy distribution to visualise.
        prior : Distribution, optional
            The prior distribution for printing the
            probability of the generated images.
        """
        x = self.model.invert(y)
        if prior is not None:
            log_py = prior.log_prob(y).view(len(x), -1).sum(-1)
            log_det = flow.log_det_jacobi(x)
            log_px = log_py + log_det
            print(f" --- first image info --- ", end='')
            print(f"log-py: {log_py[0]:.1e}, log-px: {log_px[0]:.1e}")

        x_vis = x.view(-1, 1, 28, 28).cpu()
        x_im = data_to_image(*x_vis, means=(.1307, ), stds=(.3081, ))
        display(x_im, metadata={'width': '100%'})
    
    def train(self, loader: DataLoader, num_epochs: int = 10, vis_every: int = 5,
              prior: distributions.Distribution = None):
        """
        Train a normalising flow for a number of epochs.

        Parameters
        ----------
        loader : DataLoader
            A data loader for iterating over batches of the data.
        num_epochs : int, optional
            Number of times to iterate the dataset.
        vis_every : int, optional
            Frequency, during training, of 
            intermediate visualisation of generated samples.
        prior : Distribution, optional
            The prior distribution for the normalising flow (for visualisation).
        """
        dev = self.device
        if prior is None:
            try:
                prior = self.criterion.prior
            except AttributeError:
                x, _ = next(iter(loader))
                mean = torch.zeros_like(x[0]).to(dev)
                std = torch.ones_like(x[0]).to(dev)
                prior = distributions.Normal(mean, std)

        y_vis = prior.sample((10, )).to(dev)

        errs = self.evaluate(loader, self.criterion)
        print(f"Epoch {0:02d} - avg loss: {sum(errs) / len(errs)}")
        self.display_inverted(y_vis, prior)

        for i in range(1, num_epochs + 1):
            errs = self.update(loader)
            print(f"Epoch {i:02d} - avg loss: {sum(errs) / len(errs)}")
            if i % vis_every == 0:
                self.display_inverted(y_vis, prior)

## Functions and Distributions

One of the foundations of normalising flows is the *change of variable* formula,
which allows to reason about the effect functions have on distributions.
Given a random variable $Y = f(X)$, where $f$ is a bijective function
and $X$ is a random variable from a distribution with pdf $p_X$,
the change of variable allows us to write down the pdf, $p_Y$,
that defines the distribution of $Y$.

For a scalar function $f : \mathbb{R} \to \mathbb{R}$,
the change of variable formula is given as follows:

$$\begin{aligned}
  p_Y(y) = p_X(f^{-1}(y)) \cdot \left|\frac{\partial f^{-1}(y)}{\partial y}\right|.
\end{aligned}$$

This formula reveals that the probability density of $Y = f(X)$
is proportional to the probability density of $X$.
Moreover, the proportion of densities is the gradient of the inverse,
or, equivalently, the inverse of the gradient, $f'(X)$.

To gain some intuition for the change of variable formula,
it might help to consider the visualisation of the histogram below.
Consider inputs distributed according to $X \sim \mathcal{U}(0, 2)$,
which is illustrated by the histogram below the $x$-axis.
To get a feeling for the distribution of $Y = X^2$,
we map each bin in the histogram to its corresponding range on the $y$-axis.

The *probability densities* are the height of each bin.
The area of each bin represents the actual probability,
and must be conserved through function application.
This means that if a bin on the $x$-axis is mapped to a smaller or larger range,
the density on the $y$-axis will proportionally grow or shrink, respecively.
Therefore, the change in probability density is proportional
to the the change of range due to the function.
For infinitesimally small bins, this "change of range" 
eventually corresponds to the derivative.

<figure>
  <figcaption style="text-align: center"> Change of Variables through the function $f(x) = x^2$ </figcaption>
  <figure style="display: inline-block; max-width: 33%; margin: 0;">
    <img alt="visualisation of change of variables formula with one histogram bin" src="data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22utf-8%22%3F%3E%0A%3Csvg%20viewBox%3D%220%200%20500%20500%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22200%22%20y1%3D%22320%22%20x2%3D%22200%22%20y2%3D%22300%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22300%22%20x2%3D%22200%22%20y2%3D%22300%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22200%22%20y%3D%22320%22%20width%3D%22100%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22300%22%20y1%3D%22320%22%20x2%3D%22300%22%20y2%3D%22100%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22100%22%20x2%3D%22300%22%20y2%3D%22100%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22150%22%20y%3D%22100%22%20width%3D%2230%22%20height%3D%22200%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cpath%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20fill%3A%20none%3B%22%20d%3D%22M%20200%20300%20C%20233.33%20300%20266.67%20233.33%20300%20100%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%22%20x1%3D%22200%22%20y1%3D%2290%22%20x2%3D%22200%22%20y2%3D%22310%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%22%20x1%3D%22190%22%20y1%3D%22300%22%20x2%3D%22410%22%20y2%3D%22300%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%3C%2Fsvg%3E" style="width: 500px" />
    <figcaption style="text-align: center">(a) 1-bin histogram</figcaption>
  </figure>
  <figure style="display: inline-block; max-width: 33%; margin: 0;">
    <img alt="visualisation of change of variables formula with two histogram bins" src="data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22utf-8%22%3F%3E%0A%3Csvg%20viewBox%3D%220%200%20500%20500%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22200%22%20y1%3D%22320%22%20x2%3D%22200%22%20y2%3D%22300%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22300%22%20x2%3D%22200%22%20y2%3D%22300%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22200%22%20y%3D%22320%22%20width%3D%2250%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22250%22%20y1%3D%22320%22%20x2%3D%22250%22%20y2%3D%22250%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22250%22%20x2%3D%22250%22%20y2%3D%22250%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22120%22%20y%3D%22250%22%20width%3D%2260%22%20height%3D%2250%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Crect%20x%3D%22249.96%22%20y%3D%22320%22%20width%3D%2250%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22300%22%20y1%3D%22320%22%20x2%3D%22300%22%20y2%3D%22100%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22100%22%20x2%3D%22300%22%20y2%3D%22100%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22160%22%20y%3D%22100%22%20width%3D%2220%22%20height%3D%22150%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cpath%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20fill%3A%20none%3B%22%20d%3D%22M%20200%20300%20C%20233.33%20300%20266.67%20233.33%20300%20100%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%22%20x1%3D%22200%22%20y1%3D%22310%22%20x2%3D%22200%22%20y2%3D%2290%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%22%20x1%3D%22190%22%20y1%3D%22300%22%20x2%3D%22410%22%20y2%3D%22300%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%3C%2Fsvg%3E" style="width: 500px" />
    <figcaption style="text-align: center">(b) 2-bin histogram</figcaption>
  </figure>
  <figure style="display: inline-block; max-width: 33%; margin: 0;">
    <img alt="visualisation of change of variables formula with four histogram bins" src="data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22utf-8%22%3F%3E%0A%3Csvg%20viewBox%3D%220%200%20500%20500%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22200%22%20y1%3D%22320%22%20x2%3D%22200%22%20y2%3D%22300%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22300%22%20x2%3D%22200%22%20y2%3D%22300%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22200.04%22%20y%3D%22320%22%20width%3D%2225%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22225%22%20y1%3D%22320%22%20x2%3D%22225%22%20y2%3D%22287.5%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22287.5%22%20x2%3D%22225%22%20y2%3D%22287.5%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%2260%22%20y%3D%22287.5%22%20width%3D%22120%22%20height%3D%2212.5%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Crect%20x%3D%22225%22%20y%3D%22320%22%20width%3D%2225%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22250%22%20y1%3D%22320%22%20x2%3D%22250%22%20y2%3D%22250%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22250%22%20x2%3D%22250%22%20y2%3D%22250%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22140%22%20y%3D%22250%22%20width%3D%2240%22%20height%3D%2237.5%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Crect%20x%3D%22250%22%20y%3D%22320%22%20width%3D%2225%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22275%22%20y1%3D%22320%22%20x2%3D%22275%22%20y2%3D%22187.5%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22187.5%22%20x2%3D%22275%22%20y2%3D%22187.5%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22156%22%20y%3D%22187.5%22%20width%3D%2224%22%20height%3D%2262.5%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Crect%20x%3D%22275%22%20y%3D%22320%22%20width%3D%2225%22%20height%3D%2260%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22300%22%20y1%3D%22320%22%20x2%3D%22300%22%20y2%3D%22100%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20stroke-dasharray%3A%205px%3B%22%20x1%3D%22180%22%20y1%3D%22100%22%20x2%3D%22300%22%20y2%3D%22100%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%20%20%3Crect%20x%3D%22162.9%22%20y%3D%22100%22%20width%3D%2217.14%22%20height%3D%2287.5%22%20style%3D%22stroke%3A%20rgb%28255%2C%20255%2C%20255%29%3B%20fill%3A%20rgb%2831%2C%20119%2C%20180%29%3B%22%2F%3E%0A%20%20%3Cpath%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%20fill%3A%20none%3B%22%20d%3D%22M%20200%20300%20C%20233.33%20300%20266.67%20233.33%20300%20100%22%2F%3E%0A%20%20%3Cg%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%22%20x1%3D%22200%22%20y1%3D%22310%22%20x2%3D%22200%22%20y2%3D%2290%22%2F%3E%0A%20%20%20%20%3Cline%20style%3D%22stroke%3A%20rgb%280%2C%200%2C%200%29%3B%22%20x1%3D%22190%22%20y1%3D%22300%22%20x2%3D%22410%22%20y2%3D%22300%22%2F%3E%0A%20%20%3C%2Fg%3E%0A%3C%2Fsvg%3E" style="width: 500px" />
    <figcaption style="text-align: center">(c) 4-bin histogram</figcaption>
  </figure>
</figure>

For multi-variable functions, we simply exchange the derivative
by the determinant of the Jacobian, which gives:

$$\begin{aligned}
  p_Y(\mathbf{y}) = p_X(g^{-1}(\mathbf{y})) \cdot \left|\det \mathcal{J}_g(\mathbf{y})\right|^{-1}.
\end{aligned}$$

Note that the determinant of the Jacobian corresponds to 
the change of area due to function application.
To gain some intuition for this version of the formula,
the visualisations above can be generalised to e.g. 2d histograms.

### Exercise 1:  Jacobian Check (3 points)

To get some feeling for the change of variable formula,
an invertible function with easy-to-compute Jacobian determinant has been provided.
To make sure that there are no mistakes in the provided module,
you can use the `autograd` magic in pytorch to verify its implementation.

 > Use the automatic differentiation engine of pytorch
 > to compute the Jacobian of a pytorch module.
 > The `jacobian` function should compute the Jacobian matrix
 > of a given module at a given input.
 > It should work for any PyTorch module, also if inputs and outputs are different!
 > You should **not** use `torch.autograd.jacobian` (or similar functions)
 > and your solution should support batched inputs.

In [48]:
class AffineFunction(nn.Module):
    """ An example of a very simple invertible function. """
    
    def __init__(self, shift: float = 0., log_scale: float = 0.):
        super().__init__()
        self.shift = shift
        self.log_scale = log_scale
        
    @property
    def scale(self):
        from math import exp
        return exp(self.log_scale)
    
    def forward(self, x):
        return self.scale * x + self.shift
    
    def invert(self, y):
        return (y - self.shift) / self.scale
    
    def log_det_jacobi(self, x):
        log_diag_jacobi = torch.full_like(x, self.log_scale)
        return torch.sum(log_diag_jacobi, dim=-1)


In [49]:
def jacobian(func: nn.Module, x: torch.Tensor):
    """
    Compute the Jacobian of a function at a given input.

    Parameters
    ----------
    func : nn.Module (D -> K)
        The function to compute the Jacobian for.
    x : (N, D) torch.Tensor
        Batch of inputs to compute Jacobian for.

    Returns
    -------
    Jacobian: (N, K, D)
        The computed Jacobian determinants.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    # input has gradient
    x = x.requires_grad_(True)

    # find output applying the function
    y = func(x)

    if y.dim() == 1:
        y = y.unsqueeze(-1) # if output has size 1, making sure it has 2

    n = x.shape[0] # N --> batch size
    out_dim = y.shape[1] # K --> output dimension
    in_dim = x.shape[1] # D --> input dimension

    # inizialization tensor 
    jacobian = torch.zeros(n, out_dim, in_dim) # dim N K D

    # compute jacobian

    for i in range(out_dim):
        grad_output = torch.zeros_like(y)
        grad_output[:,i] = 1 # --> take the derivative of i-th output
        y.backward(grad_output, retain_graph=True) # --> retain_graph to not delete the graph
        # collect partial derivation
        jacobian[:, i, :] = x.grad
        # zero out gradient --> no accumulation
        x.grad.zero_()

    return jacobian

In [50]:
# Test Cell: do not edit or delete!
func = AffineFunction(2, 1)
x_scalar = torch.linspace(-5, 5, 128).view(-1, 1)
j_scalar = jacobian(func, x_scalar)
assert j_scalar.shape == (128, 1, 1), (
    "ex1: jacobian output has wrong shape for scalar affine function (-1 point)"
)
assert torch.allclose(func.log_det_jacobi(x_scalar).exp(), j_scalar.squeeze()), (
    "ex1: jacobian determinant does not match log determinant of scalar affine function (-1 point)"
)

In [51]:
# Test Cell: do not edit or delete!
x_vector = torch.linspace(-5, 5, 128).view(32, 4)
j_vector = jacobian(func, x_vector)
assert j_vector.shape == (32, 4, 4), (
    "ex1: jacobian output has wrong shape for vector affine function (-1 point)"
)
assert torch.allclose(func.log_det_jacobi(x_vector).exp(), torch.det(j_vector)), (
    "ex1: jacobian determinant does not match log determinant of vector affine function (-1 point)"
)

In [52]:
# Test Cell: do not edit or delete!
func = nn.Sequential(nn.ELU(), nn.Linear(4, 3))
x = torch.linspace(-5, 5, 128).view(32, 4)
j = jacobian(func, x)
assert j.shape == (32, 3, 4), (
    "ex1: jacobian output has wrong shape for fc layer (-1 point)"
)

### Exercise 2: Change of Variables (2 points)

With an invertible module like `AffineFunction`, 
implementing the change of variables formula should not be too hard.

 > Implement the `cov_pdf` function to compute the probability density
 > of the image of a function using the change of variables formula.

In [53]:
@torch.no_grad()
def cov_pdf(y: torch.Tensor, func: nn.Module,
            x_dist: distributions.Distribution):
    """
    Compute the probability density using the change of variables formula.

    Parameters
    ----------
    y : (N, D) torch.Tensor
        Batch of values to compute probability density for.
    func : nn.Module
        The invertible function in the change of variables formula.
    x_dist : Distribution
        The distribution of the inputs.

    Returns
    -------
    p_y : (N, ) torch.Tensor
        Probability density for each value in the batch.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    # Invert y to find x
    x = func.invert(y)

    # Compute probability density at x
    p_x = x_dist.log_prob(x)  # This returns the log probability for each component
    if x.dim() > 1:
        p_x = p_x.sum(dim=-1)  # Sum log probabilities across dimensions

    log_det_jacobian = torch.full_like(p_x, -func.log_scale * y.size(-1))

    log_p_y = p_x + log_det_jacobian
    p_y = log_p_y.exp() # --> p_x(y) = exp(log p_x(x) + log df-1/dy) = px_(x) * df-1/dy

    return p_y
     


In [54]:
# Test Cell: do not edit or delete!
func = AffineFunction(2, 1)
z_gauss_1d = distributions.Normal(0., 1.)
ref_gauss_1d = distributions.Normal(func.shift, func.scale)

x_scalar = torch.linspace(-1., 1., 128).view(128, 1)
p_cov = cov_pdf(x_scalar, func, z_gauss_1d)
assert torch.allclose(p_cov, ref_gauss_1d.log_prob(x_scalar).exp()[:, 0]), (
    "ex2: cov_pdf outputs incorrect density for 1d affine function (-0.5 points)"
)

In [55]:
# Test Cell: do not edit or delete!

In [56]:
# Test Cell: do not edit or delete!

In [57]:
# Test Cell: do not edit or delete!
func = AffineFunction(2, 1)
z_mean, z_std = torch.zeros(4), torch.ones(4)
z_gauss_4d = distributions.Normal(z_mean, z_std)
ref_mean, ref_cov = z_mean + func.shift, func.scale ** 2 * torch.eye(4)
ref_gauss_4d = distributions.MultivariateNormal(ref_mean, ref_cov)

x_vector = torch.linspace(-1., 1., 128).view(32, 4)
p_cov = cov_pdf(x_vector, func, z_gauss_4d)
assert torch.allclose(p_cov, ref_gauss_4d.log_prob(x_vector).exp()), (
    "ex2: cov_pdf outputs incorrect density for 4d affine function (-0.5 points)"
)

# Normalising Flows

Given some input distribution and an invertible function,
the change of variables formula gives the distribution of the outputs.
Equivalently, it can be used to get the distribution of the inputs,
given the output distribution:

$$\begin{aligned}
  p_X(\mathbf{x}) = p_Y(g(\mathbf{x})) \cdot \left|\det \mathcal{J}_g(\mathbf{x})\right|.
\end{aligned}$$

Instead of fixing the invertible function in the formula, 
it is also possible to fix the input and output distributions.
With this mindset, the change of variables formula
provides a way to learn a function that is able to
transform one distribution to some other distribution.

Normalising flows are parameterised invertible functions
that can be used to learn a function that can transform a distribution.
More concretely, they map a complex data distribution, $X$,
to some very simple target distribution, $Y$.
To make normalising flows a practical generative model, 
two key entities should be easy to compute:
 1. the inverse: 
    To generate new data, we sample from the simple distribution 
    and use the inverse of the normalising flow to transform it 
    to a sample in the data distribution.
 2. the (log-)determinant of the Jacobian:
    Normalising flows are trained to maximise the likelihood (see below),
    which can be computed using the change of variables formula.
    The logarithm stems from the fact that the log-likelihood
    has some practical advantages over the plain likelihood.

### Exercise 3: Invertible Networks (5 points)

Neural networks are in general not (easily) invertible,
and therefore not suited for normalising flows.
One solution is to make use of so-called *coupling layers*.
A coupling layer simply splits its inputs in two parts
and computes the two parts of the outputs separately.
One part of the output is simply one of the input parts.
This input part is also used as input to a *conditioner network*.
The conditioner network computes the parameters for an invertible transformation
that is used to compute the second part of the output from the other input part.

 > Implement the `AdditiveCoupling` module below
 > so that it implements an invertible coupling layer.
 > Use $\tau(x\mathbin{;}\theta) = x + \theta$ for the invertible transformation.
 > Apart from the forward pass, you will also need to implement
 > the inverse function and the logarithm of
 > the absolute determinant of the jacobian.
 
**Hint:** You want to make sure that the identity-part of the coupling does not always operate on the same pixels (when stacking multiple coupling layers)

In [58]:
class AdditiveCoupling(nn.Module):
    """ Coupling layer with additive coupling law. """
    
    def __init__(self, sub_net: nn.Module):
        """
        Parameters
        ----------
        sub_net : nn.Module
            The sub network to use as conditioner.
        """
        super().__init__()
        self.conditioner = sub_net
    
    def forward(self, x):
        """ Compute forward pass + log determinant of jacobian. """
        y = self._forward(x)
        ldj = self.log_det_jacobi(x)
        return y, ldj
    
    def _forward(self, x):
        """ Convert inputs to outputs. """
        # YOUR CODE HERE
        # raise NotImplementedError()
        x1, x2 = x.chunk(2, dim=1)  
        
        theta = self.conditioner(x1)
        
        y2 = x2 + theta
        
        y = torch.cat((x1, y2), dim=1)
        return y
        
    def invert(self, y):
        """ Convert outputs to inputs. """
        # YOUR CODE HERE
        # raise NotImplementedError()
        y1, y2 = y.chunk(2, dim=1) 
        
        theta = self.conditioner(y1)
        
        x2 = y2 - theta
        
        x = torch.cat((y1, x2), dim=1)
        return x
    
    def log_det_jacobi(self, x):
        """ Compute logarithm of Jacobian determinant. """
        # YOUR CODE HERE
        # raise NotImplementedError()
        return torch.zeros(x.shape[0], device=x.device)

In [59]:
# Test Cell: do not edit or delete!
x = torch.ones(7, 40)
func = AdditiveCoupling(nn.Linear(20, 20))
y = func._forward(x)
assert y.shape == (7, 40), (
    "ex3: AdditiveCoupling._forward method produces outputs with incorrect shape (-0.5 points)"
)

In [60]:
# Test Cell: do not edit or delete!

In [61]:
# Test Cell: do not edit or delete!

In [62]:
# Test Cell: do not edit or delete!
round_trip = func.invert(y)
assert round_trip.shape == (7, 40), (
    "ex3: AdditiveCoupling.invert method produces outputs with incorrect shape (-0.5 points)"
)

In [63]:
# Test Cell: do not edit or delete!
assert torch.allclose(x, round_trip), (
    "ex3: AdditiveCoupling.invert does not implement inverse operation (-1 point)"
)

In [64]:
# Test Cell: do not edit or delete!

In [65]:
# Test Cell: do not edit or delete!

In [66]:
# Test Cell: do not edit or delete!
x = torch.ones(7, 40)
ldj = func.log_det_jacobi(x)
assert ldj.shape == (7, ), (
    "ex3: AdditiveCoupling.log_det_jacobi produces output with incorrect shape (-0.5 points)"
)

In [67]:
# Test Cell: do not edit or delete!

### Exercise 4: Likelihood Maximisation (3 points)

Adding the parameters of the normalising flow 
in the change of variable formula, gives the following expression

$$\begin{aligned}
  p_X(\mathbf{x} \mathbin{;} \mathbf{w}) = p_Y(g(\mathbf{x} \mathbin{;} \mathbf{w})) \cdot \left|\det \mathcal{J}_g(\mathbf{x} \mathbin{;} \mathbf{w})\right|.
\end{aligned}$$

This is exactly the probability of the data, 
given the parameters, or the *likelihood*, of the normalising flow.
By maximising this likelihood, we obtain the function
that is most likely to be the true transformation to go from $X$ to $Y$.
Note that maximising the likelihood is equivalent to 
minimising the negative logarithm of the likelihood.
This gives us the objective for training a normalising flow.

 > Complete the implementation of the `NegativeLogLikelihood` module
 > to compute the *negative log-likelihood* for a normalising flow.
 > Use the `reduction` method to combine the results 
 > from multiple samples in a batch.
 > Also, distinguish between sample density and pixel densities.

In [68]:
class NegativeLogLikelihood(nn.Module):
    """ Negative Log-Likelihood of normalising flow. """
    
    def __init__(self, prior: distributions.Distribution,
                 reduction: str = None):
        """
        Parameters
        ----------
        prior : Distribution
            The output distribution of the normalising flow.
        reduction : str, optional
            How to aggregate results for multiple samples in a batch.
        """
        super().__init__()
        self.prior = prior
        
        if reduction is None or reduction == 'mean':
            self.reduction = torch.mean
        elif reduction == 'sum':
            self.reduction = torch.sum
        elif reduction == 'none':
            self.reduction = lambda x, **kwargs: x
        else:
            raise ValueError(f"unknown reduction: '{reduction}'")
        
    def forward(self, outputs):
        """
        Compute negative log-likelihood on batch of NF outputs.

        Parameters
        ----------
        outputs : ((N, D) torch.Tensor, (N, ) torch.Tensor)
            Batch of outputs from the normalising flow.

        Returns
        -------
        nll : torch.Tensor
            The aggregated negative log-likelihood for this batch.
        """
        # YOUR CODE HERE
        # raise NotImplementedError()
        transformed_data, log_det_jacobian = outputs

        log_probs = self.prior.log_prob(transformed_data)

        log_det_jacobian = log_det_jacobian.view(-1, 1).expand_as(log_probs)

        nll = -log_probs - log_det_jacobian

        nll = nll.sum(dim=1)  
        return self.reduction(nll)

In [69]:
# Test Cell: do not edit or delete
y = torch.randn(3, 10)
nll = NegativeLogLikelihood(distributions.Normal(0., 1.), reduction="none")
loss = nll((y, torch.ones(len(y))))
assert loss.shape == (3, ), (
    "ex4: NegativeLogLikelihood produces outputs with wrong shape for none reduction (-1 point)"
)
assert torch.all(loss > 0), (
    "ex4: NegativeLogLikelihood produces non-positive outputs for steep function (-1 point)"
)

In [70]:
# Test Cell: do not edit or delete
nll = NegativeLogLikelihood(distributions.Normal(0., 1.), reduction="mean")
loss = nll((y, torch.ones(len(y))))
assert loss.shape == (), (
    "ex4: NegativeLogLikelihood produces outputs with wrong shape for mean reduction (-0.5 points)"
)
nll = NegativeLogLikelihood(distributions.Normal(0., 1.), reduction="sum")
loss = nll(func(x))
assert loss.shape == (), (
    "ex4: NegativeLogLikelihood produces outputs with wrong shape for sum reduction (-0.5 points)"
)

In [71]:
# Test Cell: do not edit or delete
nll1 = NegativeLogLikelihood(distributions.Normal(0., 1.))
nll2 = NegativeLogLikelihood(distributions.Normal(0., 2.))
assert torch.any(nll1((y, torch.ones(len(y)))) != nll2((y, torch.ones(len(y))))), (
    "ex4: NegativeLogLikelihood ignores prior (-0.5 points)"
)
assert torch.any(nll1((y, torch.ones(len(y)))) != nll1((y, torch.zeros(len(y))))), (
    "ex4: NegativeLogLikelihood ignores log_det (-0.5 points)"
)

In [72]:
# Test Cell: do not edit or delete

### Exercise 5: Training a Normalising Flow (2 points)

With the invertible network layers and objective function,
we have everything we need to train a normalising flow.
Since multi-layer networks are more powerful,
you will find the `NormalisingFlow` class below useful.

 > Construct a normalising flow with multiple coupling layers.
 > Each coupling layer should have a multi-layer conditioner.
 > The training results should produce reasonably looking images.
 

In [73]:
class NormalisingFlow(nn.Sequential):
    """ Sequence of invertible layers, forming a normalising flow. """
    
    def forward(self, x):
        """ Compute forward pass + log determinant of jacobian. """
        ld = 0
        for m in self:
            ld = ld + m.log_det_jacobi(x)
            x = m._forward(x)
        return x, ld
    
    def _forward(self, x):
        """ Convert inputs to outputs. """
        for m in self:
            x = m._forward(x)
        return x
    
    def invert(self, y):
        """ Convert outputs to inputs. """
        for m in reversed(self):
            y = m.invert(y)
        return y
    
    def log_det_jacobi(self, x):
        """ Compute logarithm of Jacobian determinant. """
        ld = 0
        for m in self:
            ld = ld + m.log_det_jacobi(x)
        return ld

In [74]:
def build_multi_flow(num_in: int = 784):
    """
    Create a normalizing flow with multiple multi-layer coupling layers.
    
    Parameters
    ----------
    num_in : int
        Number of input neurons.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    modules = []
    num_layers = 5  # Define the number of coupling layers
    
    # Define the input dimension for each half when split in coupling layers
    num_half = num_in // 2
    
    for _ in range(num_layers):
        # Each conditioner is a simple MLP with one hidden layer
        conditioner = nn.Sequential(
            nn.Linear(num_half, 256),  # Hidden layer
            nn.ReLU(),
            nn.Linear(256, num_half)   # Output layer with no non-linearity
        )
        # Append additive coupling layer with the defined conditioner
        modules.append(AdditiveCoupling(conditioner))
    
    # Return a NormalisingFlow containing all coupling layers
    return NormalisingFlow(*modules)
    

In [75]:
# Test Cell: do not edit or delete!
flow = build_multi_flow()

In [76]:
# Test Cell: do not edit or delete!

In [77]:
# Test Cell: do not edit or delete!
x = torch.randn(3, 784)
y = flow._forward(x)
assert y.shape == x.shape, (
    "ex5: build_multi_flow does not produce outputs with correct shape (-1 point)"
)

In [78]:
# data
pre_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.1307, ), (.3081, )),
    nn.Flatten(0)
])
mnist = datasets.MNIST(data_root, transform=pre_process)
loader = DataLoader(mnist, batch_size=128, shuffle=True)

# objective
mean = torch.zeros(784).to(device)
std = torch.ones(784).to(device)
prior = distributions.Normal(mean, std)
nll = NegativeLogLikelihood(prior)

In [79]:
flow = build_multi_flow().to(device)
trainer = NormalisingFlowTrainer(
    model=flow,
    criterion=nll,
    optimiser=optim.Adam(flow.parameters(), lr=1e-3)
)

trainer.train(loader, num_epochs=9, vis_every=3)

Epoch 00 - avg loss: 1166.066595732276
 --- first image info --- log-py: -1.1e+03, log-px: -1.1e+03


Epoch 01 - avg loss: 994.135522773271
Epoch 02 - avg loss: 981.2256849732227
Epoch 03 - avg loss: 977.8521590568364
 --- first image info --- log-py: -1.1e+03, log-px: -1.1e+03


Epoch 04 - avg loss: 975.9228415418027
Epoch 05 - avg loss: 974.3897421375267
Epoch 06 - avg loss: 973.1891186679604
 --- first image info --- log-py: -1.1e+03, log-px: -1.1e+03


Epoch 07 - avg loss: 972.3170285743437
Epoch 08 - avg loss: 971.478317163138
Epoch 09 - avg loss: 970.7560643438082
 --- first image info --- log-py: -1.1e+03, log-px: -1.1e+03
